# Импорт библиотек

In [2]:
import warnings
import pandas as pd

warnings.filterwarnings("ignore")

# Подготовка данных

In [ ]:
df = pd.read_csv("../../data/frames_errors.csv", header=None)
df.columns = [
    "block_id",  # ID блока
    "frame_idx",  # ID фрейма в блоке
    "E_mu_Z",  # Факт: Реальный QBER в Z-базисе
    "E_mu_phys_est",  # Альтернативная оценка QBER, основанная на физических параметрах
    "E_mu_X",  # Факт: Реальный QBER в X-базисе
    "E_nu1_X",  # QBER для decoy-состояния nu1 в X-базисе
    "E_nu2_X",  # QBER для decoy-состояния nu2 в X-базисе
    "E_nu1_Z",  # QBER для decoy-состояния nu1 в Z-базисе
    "E_nu2_Z",  # QBER для decoy-состояния nu2 в Z-базисе
    "N_mu_X",  # Количество отправленных сигнальных состояний в X-базисе
    "M_mu_XX",  # Количество принятых сигнальных состояний (отправлено в X, принято в X)
    "M_mu_XZ",  # Количество принятых сигнальных состояний (отправлено в X, принято в Z)
    "M_mu_X",  # Общее количество принятых сигнальных состояний в X-базисе
    "N_mu_Z",  # Количество отправленных сигнальных состояний в Z-базисе
    "M_mu_ZZ",  # Количество принятых сигнальных состояний (отправлено в Z, принято в Z)
    "M_mu_Z",  # Общее количество принятых сигнальных состояний в Z-базисе
    "N_nu1_X",  # Количество отправленных decoy-состояний nu1 в X-базисе
    "M_nu1_XX",
    "M_nu1_XZ",
    "M_nu1_X",
    "N_nu1_Z",
    "M_nu1_ZZ",
    "M_nu1_Z",
    "N_nu2_X",
    "M_nu2_XX",
    "M_nu2_XZ",
    "M_nu2_X",
    "N_nu2_Z",
    "M_nu2_ZZ",
    "M_nu2_Z",
    "nTot",  # Общее количество отправленных состояний
    "bayesImVoltage",  # Напряжение на модуляторе интенсивности
    "opticalPower",  # Оптическая мощность
    "polarizerVoltages[0]",  # Напряжение на контроллере поляризации 0
    "polarizerVoltages[1]",  # Напряжение на контроллере поляризации 1
    "polarizerVoltages[2]",  # Напряжение на контроллере поляризации 2
    "polarizerVoltages[3]",  # Напряжение на контроллере поляризации 3
    "temp_1",  # Температура детектора 1
    "biasVoltage_1",  # Напряжение смещения детектора 1
    "temp_2",  # Температура детектора 2
    "biasVoltage_2",  # Напряжение смещения детектора 2
    "synErr",  # Кол-во ошибок в битах
    "N_EC_rounds",  # Результат: Количество доп. раундов коррекции (индикатор сложности)
    "maintenance_flag",  # Флаг технического обслуживания
    "estimator_name",  # Название алгоритма, который выбрал параметры в реальности
    "f_EC",  # Эффективность коррекции ошибок
    "E_mu_Z_est",  # Предсказание: Оценка QBER, на основе которой были выбраны R, s, p
    "R",  # Параметр: Скорость кода
    "s",  # Параметр: Количество укороченных узлов
    "p",  # Параметр: Количество проколотых узлов
]

In [4]:
missing_counts = df.isna().sum()
columns_with_missing = missing_counts[missing_counts > 0]

if columns_with_missing.empty:
    print("Пропусков в данных нет.")
else:
    print("Столбцы с пропусками и их количество:")
    print(columns_with_missing)

Столбцы с пропусками и их количество:
E_mu_phys_est    448
f_EC             131
dtype: int64


In [5]:
timestamp_counts = df.groupby("block_id")["frame_idx"].nunique()
print("Количество фреймов / Количество рядов")
print(timestamp_counts.value_counts())

Количество фреймов / Количество рядов
frame_idx
399    569
400    251
398      2
390      1
Name: count, dtype: int64


In [6]:
# Создаем два под-датафрейма: для оптимистичных и пессимистичных прогнозов
df_optimistic = df[df["E_mu_Z_est"] < df["E_mu_Z"]]
df_pessimistic = df[df["E_mu_Z_est"] > df["E_mu_Z"]]

# Считаем среднее для каждой группы
avg_rounds_optimistic = df_optimistic["N_EC_rounds"].mean()
avg_rounds_pessimistic = df_pessimistic["N_EC_rounds"].mean()

print(f"Количество оптимистичных прогнозов (E_mu_Z_est < E_mu_Z): {len(df_optimistic)}")
print(f"Средний N_EC_rounds для них: {avg_rounds_optimistic:.4f}\n")

print(f"Количество пессимистичных прогнозов (E_mu_Z_est > E_mu_Z): {len(df_pessimistic)}")
print(f"Средний N_EC_rounds для них: {avg_rounds_pessimistic:.4f}")

Количество оптимистичных прогнозов (E_mu_Z_est < E_mu_Z): 152624
Средний N_EC_rounds для них: 9.4753

Количество пессимистичных прогнозов (E_mu_Z_est > E_mu_Z): 175417
Средний N_EC_rounds для них: 2.0533


In [7]:
pessimistic_diff = df_pessimistic["E_mu_Z_est"] - df_pessimistic["E_mu_Z"]
avg_pessimistic_overestimation = pessimistic_diff.mean()
print(
    f"В среднем, в пессимистичных случаях E_mu_Z_est больше E_mu_Z на: {avg_pessimistic_overestimation:.6f}"
)

В среднем, в пессимистичных случаях E_mu_Z_est больше E_mu_Z на: 0.001945


In [8]:
avg_pessimistic_ratio = (df_pessimistic["E_mu_Z_est"] / df_pessimistic["E_mu_Z"]).mean()

print(
    f"В среднем, в пессимистичных случаях E_mu_Z_est больше E_mu_Z в: {avg_pessimistic_ratio:.4f} раз"
)

В среднем, в пессимистичных случаях E_mu_Z_est больше E_mu_Z в: 1.1023 раз


In [9]:
start_idx = df[(df["block_id"] == 1489460492) & (df["frame_idx"] == 99)].index[0]
end_idx = df[(df["block_id"] == 1840064900) & (df["frame_idx"] == 101)].index[0]

df_submit = df.loc[start_idx:end_idx].copy()

In [10]:
from math import ceil

alpha = 0.33
f_ec = 1.15
R_range = [
    round(0.50 + 0.05 * x, 2) for x in range(9)
]  # 0.50..0.90 для соответствия условию задачи
n = 32000
d = 4800

In [11]:
import numpy as np


def calculate_ema(prev_ema, current_value, alpha):
    if prev_ema is None:
        return current_value
    return alpha * current_value + (1 - alpha) * prev_ema


def h(x):
    if x > 0:
        return -x * np.log2(x) - (1 - x) * np.log2(1 - x)
    elif x == 0:
        return 0.0
    else:
        raise ValueError("Invalid x for binary entropy")


def select_code_rate(e_mu, f_ec, rates, frame_len, sp_count):
    r_candidate = 1 - h(e_mu) * f_ec
    R_res = 0.50
    s_n = sp_count
    p_n = 0
    for R in rates:
        p_n = int(ceil((1 - R) * frame_len - (1 - r_candidate) * (frame_len - sp_count)))
        s_n = int(sp_count - p_n)
        if p_n >= 0 and s_n >= 0:
            R_res = R
            return round(R_res, 2), s_n, p_n
    return round(R_res, 2), s_n, p_n

In [12]:
E_series = (
    pd.to_numeric(df_submit["E_mu_Z"] * 1.1023, errors="coerce").dropna().reset_index(drop=True)
)

prev_ema = None
rows = []
for E_mu_Z in E_series:
    ema_value = calculate_ema(prev_ema, float(E_mu_Z), alpha)
    prev_ema = ema_value
    R, s_n, p_n = select_code_rate(ema_value, f_ec, R_range, n, d)
    rows.append([f"{E_mu_Z:.16f}", R, s_n, p_n])  # 4 столбца: E, R, s_n, p_n

In [13]:
pd.DataFrame(rows).to_csv("submission.csv", header=False, index=False)

In [14]:
my_df = pd.DataFrame(rows).iloc[:, 0].astype(float)
other_df = pd.read_csv("submission_oleg(2).csv", header=None).iloc[:, 0].astype(float)

In [15]:
other_df.mean()

0.02103137568213607

In [16]:
my_df.mean()

0.021920591328999997